# 📊 Notebook: Modelos de Machine Learning - Forecasting TJGO

Este notebook demonstra o uso de modelos de machine learning para previsão de casos no TJGO:
- Random Forest
- XGBoost  
- LightGBM
- Modelos Baseline

## 🎯 Objetivos
- Treinar modelos de ML com features temporais
- Analisar importância das features
- Comparar performance entre modelos
- Avaliar resíduos e diagnósticos

## 📈 Características dos Modelos ML
- **Random Forest**: Ensemble de árvores, robusto a overfitting
- **XGBoost**: Gradient boosting otimizado, alta performance
- **LightGBM**: Gradient boosting eficiente, rápido treinamento
- **Features**: Lags, rolling statistics, variáveis exógenas


In [ ]:
# Importações necessárias
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

# Importar modelos de ML
import sys
sys.path.append('../src')
from models.ml_models import (
    RandomForestModel, XGBoostModel, LightGBMModel, BaselineModels,
    train_random_forest_model, train_xgboost_model, train_lightgbm_model,
    train_baseline_models
)

# Configurar matplotlib
plt.style.use('seaborn-v0_8')
plt.rcParams['figure.figsize'] = (12, 6)

print("✅ Bibliotecas importadas com sucesso!")
print("📊 Pronto para usar os modelos de ML")


In [ ]:
# Carregar dados processados
train_data = pd.read_csv('../data/processed_test/train_test.csv', index_col='DATA', parse_dates=True)
test_data = pd.read_csv('../data/processed_test/test_test.csv', index_col='DATA', parse_dates=True)

print("📊 Dados carregados:")
print(f"  Treino: {len(train_data)} observações")
print(f"  Teste:  {len(test_data)} observações")
print(f"  Variáveis: {len(train_data.columns)}")

# Visualizar features disponíveis
print("\n📋 Features disponíveis:")
feature_cols = train_data.select_dtypes(include=[np.number]).columns.tolist()
for i, col in enumerate(feature_cols):
    print(f"  {i+1:2d}. {col}")


## 🔧 2. Treinamento dos Modelos


In [ ]:
# Treinar modelos baseline
print("🔄 Treinando modelos baseline...")
baseline_models = train_baseline_models(train_data, test_data)


In [ ]:
# Treinar Random Forest
print("\n🔄 Treinando Random Forest...")
rf_model = train_random_forest_model(train_data, test_data)
rf_model.print_summary("Random Forest")


In [ ]:
# Treinar XGBoost
print("\n🔄 Treinando XGBoost...")
xgb_model = train_xgboost_model(train_data, test_data)
xgb_model.print_summary("XGBoost")


In [ ]:
# Treinar LightGBM
print("\n🔄 Treinando LightGBM...")
lgb_model = train_lightgbm_model(train_data, test_data)
lgb_model.print_summary("LightGBM")


## 📈 3. Análises e Visualizações


In [ ]:
# Plotar previsões de todos os modelos
fig, axes = plt.subplots(2, 2, figsize=(15, 12))
axes = axes.ravel()

models = [
    (baseline_models.baseline_results['persistence'], 'Baseline Persistência'),
    (baseline_models.baseline_results['moving_average'], 'Baseline Média Móvel'),
    (rf_model, 'Random Forest'),
    (xgb_model, 'XGBoost')
]

for i, (model_data, name) in enumerate(models):
    ax = axes[i]
    
    if isinstance(model_data, dict):
        predictions = model_data['predictions']
        metrics = model_data['metrics']
        mae = metrics['mae']
    else:
        predictions = model_data.predictions
        mae = model_data.metrics['mae']
    
    # Plotar dados reais e previsões
    ax.plot(test_data.index, test_data['TOTAL_CASOS'], 
           label='Real', linewidth=2, color='blue')
    ax.plot(test_data.index, predictions, 
           label='Previsão', linewidth=2, color='red', linestyle='--')
    
    ax.set_title(f"{name} (MAE: {mae:.0f})", fontweight='bold')
    ax.set_xlabel('Data')
    ax.set_ylabel('TOTAL_CASOS')
    ax.legend()
    ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('../reports_test/ml_models_comparison.png', dpi=300, bbox_inches='tight')
plt.show()


In [ ]:
# Analisar importância das features (Random Forest)
rf_model.plot_feature_importance("Random Forest", top_n=10, 
                                save_path="../reports_test/rf_feature_importance.png")


In [ ]:
# Analisar importância das features (XGBoost)
xgb_model.plot_feature_importance("XGBoost", top_n=10, 
                                  save_path="../reports_test/xgb_feature_importance.png")


In [ ]:
# Plotar resíduos dos modelos
rf_model.plot_residuals("Random Forest", save_path="../reports_test/rf_residuals.png")
xgb_model.plot_residuals("XGBoost", save_path="../reports_test/xgb_residuals.png")
